In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.chdir("..")
os.getcwd()

'C:\\Users\\acmiy\\OneDrive\\Documents\\cs229\\cs229-f19-wiki-forecast'

In [3]:
import sys
from pathlib import Path
sys.path.append(str(Path(".").resolve() / "wikicast"))

import numpy as np
from wikicast import poisson

# regression  on random poisson data

In [4]:
def rmse(y, y_pred):
    return np.sqrt(np.sum(y-y_pred)**2)

def mape(y, y_pred):
    return np.sum(np.ma.divide(np.abs(y-y_pred), np.abs(y)).filled(0))*100

def summarize(validate, train, test):
    print("validate", rmse(validate, train), mape(validate, train))
    print("test", rmse(test, validate), mape(test, validate))

16 time series, with a window size k=4

In [5]:
# k = 4
#t = lambda i: k * i
Y = np.random.poisson(3, (16, 16))
t = lambda i: [np.newaxis,4, 5, 6][i]
train, validate, test = Y[t(0):t(1)].T, Y[t(1):t(2)].T, Y[t(2):t(3)].T
train.shape, validate.shape, test.shape

((16, 4), (16, 1), (16, 1))

In [6]:
poisson_reg = poisson.PoissonRegression()

# shift right

In [7]:
print("validate", rmse(validate, train), mape(validate, train))
print("test", rmse(test, validate), mape(test, validate))

validate 34.0 3076.6666666666665
test 9.0 1456.6666666666667


# mean

In [8]:
print("validate", rmse(validate, train.mean(axis=0)), mape(validate, train.mean(axis=0)))
z = np.hstack([train[:,1:], validate])
print("test", rmse(test, z.mean(axis=0)), mape(test, z.mean(axis=0)))

validate 34.0 1770.4166666666665
test 16.0 3955.833333333334


# evaluate on validation fit

In [9]:
poisson_reg.fit(train, validate)
y_pred = poisson_reg.predict(train)
print("validate", rmse(validate, y_pred), mape(validate, y_pred))
print("test", rmse(test, y_pred), mape(test, y_pred))

n_examples:16 d:4
theta= [0.08613002 0.05349662 0.10988867 0.10249348] delta= 0.00019990911262607132 iter= 250
theta= [0.09493299 0.037159   0.12305724 0.09935528] delta= 0.00014175596638551867 iter= 500
theta= [0.10082269 0.02465308 0.13187786 0.09611348] delta= 0.00010407880722869017 iter= 750
theta= [0.10516735 0.01546103 0.13828669 0.09365528] delta= 7.669949627591203e-05 iter= 1000
theta= [0.1083896  0.00867983 0.14296811 0.09180709] delta= 5.667848566497574e-05 iter= 1250
theta= [0.11078783 0.00366157 0.14639907 0.09042493] delta= 4.196654675883913e-05 iter= 1500
theta= [ 1.12577630e-01 -6.12892138e-05  1.48918980e-01  8.93958763e-02] delta= 3.1116945065152714e-05 iter= 1750
theta= [ 0.11391632 -0.00282875  0.15077202  0.08863283] delta= 2.309463380305509e-05 iter= 2000
theta= [ 0.11491954 -0.00488953  0.1521353   0.0880693 ] delta= 1.7151495495977374e-05 iter= 2250
theta= [ 0.11567271 -0.00642639  0.15313805  0.08765486] delta= 1.2742592690055134e-05 iter= 2500
theta= [ 0.116148

# predict with validation data (left shift)

In [10]:
z = np.hstack([train[:,:-1], validate])
y_pred = poisson_reg.predict(z)
print("validate", rmse(validate, y_pred), mape(validate, y_pred))
print("test", rmse(test, y_pred), mape(test, y_pred))

validate 108.18946664657554 8642.68418194005
test 35.810533353424475 16552.070179747814


# retrain on validation predictions

In [11]:
poisson_reg.fit(z, poisson_reg.predict(z))
y_pred = poisson_reg.predict(z)
print("validate", rmse(validate, y_pred), mape(validate, y_pred))
print("test", rmse(test, y_pred), mape(test, y_pred))

n_examples:16 d:4
theta= [0.08799612 0.05508049 0.10946654 0.101893  ] delta= 0.00020663746442583226 iter= 250
theta= [0.0942653  0.04067497 0.12143993 0.10197146] delta= 0.00011445360104090342 iter= 500
theta= [0.09824159 0.029015   0.12910606 0.09997131] delta= 8.917999872574042e-05 iter= 750
theta= [0.10154689 0.02032284 0.13488613 0.09800376] delta= 6.968931331802291e-05 iter= 1000
theta= [0.10428823 0.01380394 0.13927482 0.09620776] delta= 5.456298281612375e-05 iter= 1250
theta= [0.10654831 0.00888403 0.14262024 0.09463251] delta= 4.277567055269905e-05 iter= 1500
theta= [0.10840125 0.00515014 0.14517858 0.09328684] delta= 3.356442672279183e-05 iter= 1750
theta= [0.10991294 0.00230242 0.14714018 0.09215847] delta= 2.6352333599676794e-05 iter= 2000
theta= [1.11140938e-01 1.21181551e-04 1.48647522e-01 9.12251550e-02] delta= 2.0698047194370598e-05 iter= 2250
theta= [ 0.11213484 -0.00155586  0.14980788  0.09046114] delta= 1.6261115065473475e-05 iter= 2500
theta= [ 0.11293676 -0.0028495

# forecast horizon

In [12]:
# vector predictions
k = 4
t = lambda i: k * i
train, validate, test = Y[t(0):t(1)].T, Y[t(1):t(2)].T, Y[t(2):t(3)].T
train.shape, validate.shape, test.shape

((16, 4), (16, 4), (16, 4))

In [13]:
poisson_reg.fit(train, validate)
y_pred = poisson_reg.predict(train)
print("validate", rmse(validate, y_pred), mape(validate, y_pred))
print("test", rmse(test, y_pred), mape(test, y_pred))

n_examples:16 d:4


ValueError: cannot reshape array of size 64 into shape (16,1)